In [8]:
import pandas as pd
import numpy as np
from itertools import product
from datetime import datetime
import category_encoders as ce

# Define the range for dates in 2025
start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Define the weekdays in Spanish
dias = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']

# Map dates to corresponding weekdays
dates = [(fecha.strftime('%Y-%m-%d'), dias[fecha.weekday()]) for fecha in date_range]

# Define exhaustive lists for each other column
horas = np.arange(0.0, 24.0, 1.0)    # Decimal hours from 0.0 to 23.0
id_peaje = ['ALB', 'AVE', 'DEC', 'ILL', 'RET', 'SAL', 'SAR']
sentido = ['Centro', 'Provincia']
tipo_vehiculo = ['Liviano', 'Pesado']
tipo_cobro = ['Tag', 'Efectivo', 'Monedero virtual', 'Rec. Deuda', 'Exento', 'Tarjeta Magnetica', 'Infraccion']

# Create all combinations of other columns
combinations = list(product(horas, id_peaje, sentido, tipo_vehiculo, tipo_cobro))

# Expand each date to all combinations of other columns
data = []
for fecha, dia in dates:
    for combo in combinations:
        data.append((fecha, dia, *combo))

# Create DataFrame
columns = ['fecha', 'dia', 'hora', 'id_peaje', 'sentido', 'tipo_vehiculo', 'tipo_cobro']
df = pd.DataFrame(data, columns=columns)

# Display part of the DataFrame to confirm
print(df.head())
print(f"DataFrame size: {df.shape}")

df.to_csv("2025test1.csv", index=False)

        fecha        dia  hora id_peaje sentido tipo_vehiculo  \
0  2025-01-01  Miercoles   0.0      ALB  Centro       Liviano   
1  2025-01-01  Miercoles   0.0      ALB  Centro       Liviano   
2  2025-01-01  Miercoles   0.0      ALB  Centro       Liviano   
3  2025-01-01  Miercoles   0.0      ALB  Centro       Liviano   
4  2025-01-01  Miercoles   0.0      ALB  Centro       Liviano   

         tipo_cobro  
0               Tag  
1          Efectivo  
2  Monedero virtual  
3        Rec. Deuda  
4            Exento  
DataFrame size: (1716960, 7)


In [9]:
def prepare_dataset(filepath):
    """
    Carga y preprocesa el dataset de peajes.
    """

    dataset = pd.read_csv(filepath)

    # Se separa el día y el mes del año en columnas separadas
    # Necesitamos que las variables sean numéricas
    dataset["nro_dia"] = pd.to_datetime(dataset["fecha"]).dt.day
    dataset["nro_mes"] = pd.to_datetime(dataset["fecha"]).dt.month
    dataset["anio"] = pd.to_datetime(dataset["fecha"]).dt.year
    dataset.drop("fecha", axis=1, inplace=True)

    return dataset

def encode_columns(dataset):
    """
    Codifica las columnas del dataset que tienen variables cualitativas.
    """

    dataset = dataset.copy()

    # Se codifica la categoría día de la semana con label encoding (codificación de etiquetas)
    # Se toma esta decisión porque la variable es ordinal
    day_encoding = {
        'Lunes': 0,
        'Martes': 1,
        'Miercoles': 2,
        'Jueves': 3,
        'Viernes': 4,
        'Sabado': 5,
        'Domingo': 6
    }

    dataset['dia_semana'] = dataset['dia'].map(day_encoding)
    dataset.drop("dia", axis=1, inplace=True)

    # Se codifica el resto de las variables categóricas con binary encoding (codificación binaria)
    binary_encoder = ce.BinaryEncoder(cols=["id_peaje", "sentido", "tipo_vehiculo", "tipo_cobro"])
    dataset = binary_encoder.fit_transform(dataset)
    # Eliminamos las columnas sentido_1 y tipo_vehiculo_1 porque no aportan información
    dataset.drop(["sentido_1", "tipo_vehiculo_1"], axis=1, inplace=True)

    # Cambiarle el nombre a las columnas sentido y tipo_vehículo para saber qué significa cada valor
    dataset.rename(columns={'sentido_0': 'hacia_provincia'}, inplace=True)
    dataset.rename(columns={'tipo_vehiculo_0': 'es_pesado'}, inplace=True)

    return dataset

dataset_original = prepare_dataset("2025test1.csv")
dataset = encode_columns(dataset_original)
dataset.info()
dataset.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716960 entries, 0 to 1716959
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   hora             float64
 1   id_peaje_0       int64  
 2   id_peaje_1       int64  
 3   id_peaje_2       int64  
 4   hacia_provincia  int64  
 5   es_pesado        int64  
 6   tipo_cobro_0     int64  
 7   tipo_cobro_1     int64  
 8   tipo_cobro_2     int64  
 9   nro_dia          int32  
 10  nro_mes          int32  
 11  anio             int32  
 12  dia_semana       int64  
dtypes: float64(1), int32(3), int64(9)
memory usage: 150.6 MB


,hora,id_peaje_0,id_peaje_1,id_peaje_2,hacia_provincia,es_pesado,tipo_cobro_0,tipo_cobro_1,tipo_cobro_2,nro_dia,nro_mes,anio,dia_semana
1415923,0.0,0,0,1,1,0,1,1,0,29,10,2025,2
834073,7.0,1,0,0,0,1,0,1,1,27,6,2025,4
1270650,2.0,1,1,1,0,1,1,0,0,28,9,2025,6
304650,18.0,0,1,1,0,1,1,0,0,6,3,2025,3
371839,1.0,0,0,1,1,1,1,1,1,21,3,2025,4
739676,5.0,1,1,1,0,0,0,0,1,7,6,2025,5
459644,17.0,0,0,1,1,1,1,0,0,8,4,2025,1
76314,5.0,0,1,1,1,0,0,0,1,17,1,2025,4
832510,23.0,1,0,0,1,0,0,0,1,26,6,2025,3
684912,14.0,1,0,0,0,0,1,0,1,26,5,2025,0
